# 구글 드라이브 연동


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 필요한 라이브러리 설치

In [4]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-faitv7v6
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-faitv7v6
     |████████████████████████████████| 132 kB 7.8 MB/s 
     |████████████████████████████████| 344 kB 54.9 MB/s 
     |████████████████████████████████| 47.3 MB 45.1 MB/s 
     |████████████████████████████████| 4.5 MB 46.9 MB/s 
     |████████████████████████████████| 1.2 MB 49.9 MB/s 
     |████████████████████████████████| 4.0 MB 46.3 MB/s 
     |████████████████████████████████| 6.5 MB 42.2 MB/s 
     |████████████████████████████████| 596 kB 50.9 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 50.1 MB/s 
     |████████████████████████████████| 8.7 MB 49.3 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 127 kB 58.7 MB/s 
  Created wheel for kobert: filename=ko

In [6]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 542 kB/s 
     |████████████████████████████████| 448 kB 56.0 MB/s 


# 데이터불러오기 

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [11]:
import pandas as pd

train_data = pd.read_csv('/content/drive/MyDrive/통계데이터_인공지능_활용대회/1. 실습용자료.txt', sep = "|", encoding = "cp949")
test_data = pd.read_csv('/content/drive/MyDrive/통계데이터_인공지능_활용대회/2. 모델개발용자료.txt', sep = "|", encoding = "cp949")


In [12]:
class_spread = train_data['digit_3'].value_counts()
class_name = train_data['digit_3'].unique()

In [13]:
print('클래스 개수는 : ',len(class_name))
print('class_name: \n',class_name)

클래스 개수는 :  225
class_name: 
 [952 472 467 475 872 291 561 107 841 423 102 474 856 529 473 761 911 562
 221 961 292 477 969 949 471 551 464 493 259 732 478 912 422 222 855 320
 711 465 582 721 863 559 729 141 476 461 452 953 682 339 862 466 281 712
 492 752 203 681 901 742 713 592 340 261 463 262 411 163 424 243 521 851
 106 421 479 941 902 311 468 204  72 181 662 852 641 239 453 251 103 762
 202 151 462 653 233 161 861 101 620 733 104 412 857 134 242 661 426 132
 171 751 112 139 701 263 284 162 303 715 942 501 232  71 144 331 383 702
 871 759 741 179 241 289 591 763 951 105 869 451 172 739 611 272 271 649
 351 302 285 753 111 264 425 201 265 334 639 842 231 205 131 494 581 642
 274 764 382 651 283 319 370 333 631  20 602 390  11 743 714 133 108  32
 143 716 152 844 381 313 612 212  14 353 360 352 312 273 332 191 282  12
 252  31 731 213 301 192 182 495 211  62 511  61 853 502 854 845 512 142
 491 304  80 652 843 601 266  51 120]


In [14]:
my_dict = dict(class_spread) # 클래스 개수별 분포

In [15]:
sorted_dict = dict(sorted(my_dict.items())) # 정렬 
print(sorted_dict)

number = 11 # 원하는 클래스 
print('class {}의 개수는 '.format(number),sorted_dict[number])

{11: 396, 12: 254, 14: 177, 20: 121, 31: 13, 32: 103, 51: 1, 61: 1, 62: 6, 71: 190, 72: 223, 80: 3, 101: 729, 102: 985, 103: 1177, 104: 769, 105: 64, 106: 1353, 107: 9214, 108: 285, 111: 250, 112: 137, 120: 3, 131: 474, 132: 2819, 133: 217, 134: 965, 139: 812, 141: 4421, 142: 66, 143: 304, 144: 626, 151: 865, 152: 514, 161: 335, 162: 1344, 163: 45, 171: 206, 172: 996, 179: 552, 181: 4139, 182: 38, 191: 20, 192: 68, 201: 338, 202: 658, 203: 334, 204: 1643, 205: 56, 211: 39, 212: 156, 213: 118, 221: 803, 222: 4935, 231: 523, 232: 670, 233: 758, 239: 922, 241: 1279, 242: 533, 243: 403, 251: 4156, 252: 33, 259: 12876, 261: 391, 262: 1666, 263: 284, 264: 718, 265: 239, 266: 8, 271: 1484, 272: 1506, 273: 170, 274: 47, 281: 2756, 282: 84, 283: 412, 284: 1338, 285: 580, 289: 354, 291: 4924, 292: 7163, 301: 20, 302: 323, 303: 2407, 304: 22, 311: 732, 312: 93, 313: 106, 319: 101, 320: 3275, 331: 605, 332: 76, 333: 295, 334: 212, 339: 3396, 340: 3380, 351: 660, 352: 67, 353: 29, 360: 147, 370: 43

# 불용어 처리 


In [16]:
df_mod = train_data[['digit_3','text_obj','text_mthd','text_deal']]

In [17]:
# df_mod = df_mod.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(df_mod.isnull().values.any()) # Null 값이 존재하는지 확인
df_mod = df_mod.fillna(' ') # Nan 값을 빈 string으로 바꿈
print(df_mod.isnull().values.any()) # Null 값이 존재하는지 확인
# False => nan값이 없다

True
False


In [18]:
# data x, y 로 나누기
x1 = df_mod['text_obj'].astype(str)
x2 = df_mod['text_mthd'].astype(str)
x3 = df_mod['text_deal'].astype(str)

In [19]:
remove_1 = ['에서','으로','에게','하여',] # text_obj, text_mthd,
remove_2 =['일반','고객'] # text_mthd 
remove_3 = ['를 통하여','을 갖추고','를통하여'] #  text_mthd
for i in range(len(x1)):
  # remove 1
  if x1.iloc[i][-2::]in remove_1: # text_obj
    x1[i]=x1.iloc[i][0:-2]
  
  if x2.iloc[i][-2::]in remove_1: # text_mthd
    x2[i]=x2.iloc[i][0:-2]

  # remove 2
  if x2.iloc[i][0:2]in remove_2:  # text_mthd
    x2[i]=' ' # remove_2 들어가면 제거 
    
  # remove 3
  if x2.iloc[i][-5::]in remove_3: # text_mthd
    x2[i]=x1.iloc[i][0:-5]
  
  if x2.iloc[i][-4::]in remove_3: # text_mthd
    x2[i]=x1.iloc[i][0:-4]
    
  if i % 100000 == 0:
    print(i)
    
    
  


0
100000
200000
300000
400000
500000
600000
700000
800000
900000


In [20]:
x = x1 + ',' + x2+','+ x3
y = df_mod['digit_3']

In [21]:
# 클래스 이름, 클래스 개수 
print("class 개수는 ",len(df_mod['digit_3'].unique()), df_mod['digit_3'].unique())
class_name = df_mod['digit_3'].unique()
# class change (11~969 => 0~225)
for idx, class_n in enumerate(class_name):
  df_mod.loc[(df_mod['digit_3'] == class_n),'digit_3'] = idx

class 개수는  225 [952 472 467 475 872 291 561 107 841 423 102 474 856 529 473 761 911 562
 221 961 292 477 969 949 471 551 464 493 259 732 478 912 422 222 855 320
 711 465 582 721 863 559 729 141 476 461 452 953 682 339 862 466 281 712
 492 752 203 681 901 742 713 592 340 261 463 262 411 163 424 243 521 851
 106 421 479 941 902 311 468 204  72 181 662 852 641 239 453 251 103 762
 202 151 462 653 233 161 861 101 620 733 104 412 857 134 242 661 426 132
 171 751 112 139 701 263 284 162 303 715 942 501 232  71 144 331 383 702
 871 759 741 179 241 289 591 763 951 105 869 451 172 739 611 272 271 649
 351 302 285 753 111 264 425 201 265 334 639 842 231 205 131 494 581 642
 274 764 382 651 283 319 370 333 631  20 602 390  11 743 714 133 108  32
 143 716 152 844 381 313 612 212  14 353 360 352 312 273 332 191 282  12
 252  31 731 213 301 192 182 495 211  62 511  61 853 502 854 845 512 142
 491 304  80 652 843 601 266  51 120]


In [22]:
data_list = []
for q, label in zip(x, y)  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

# 불용어처리 재시도

In [23]:
stop_words = "에서 로 으로 갖추고 일반 대상 . , 등 따라 의해 에 로 위하여 ? 통해 하여 에게 소비 제공 소비자 고객 일반인 을 를 의 하여 해서 요청 가지고 및 두고 받아 받음 서비스"
stop_words = set(stop_words.split(' '))
print(stop_words)

{'위하여', '두고', '가지고', '하여', '갖추고', '소비자', '일반인', '소비', '에', '해서', '.', '의해', '요청', '받음', '에게', '통해', '받아', '등', '고객', '?', '을', '제공', '의', '로', ',', '따라', '대상', '으로', '및', '서비스', '를', '에서', '일반'}


In [24]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

In [25]:
okt = Okt()

In [ ]:
cnt = 0
for i in range(len(data_list)):
  q = data_list[i][0]
  word_tokens = okt.morphs(q)  
  result = ' '
  for word in word_tokens:
    if not word in stop_words:
      result += ' ' + word  
  data_list[i][0] = result
  cnt += 1
  if cnt%3000 == 0:
    print(cnt)

3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
60000
63000
66000
69000
72000
75000
78000
81000
84000
87000
90000
93000
96000
99000
102000
105000
108000
111000
114000
117000
120000
123000
126000
129000
132000
135000
138000
141000
144000
147000
150000
153000
156000
159000
162000
165000
168000
171000
174000
177000
180000
183000
186000
189000
192000
195000
198000
201000
204000
207000
210000
213000
216000
219000
222000
225000
228000
231000
234000
237000
240000
243000
246000
249000
252000
255000
258000
261000
264000
267000
270000
273000
276000
279000
282000
285000
288000
291000
294000
297000
300000
303000
306000
309000
312000
315000
318000
321000
324000
327000
330000
333000
336000
339000
342000
345000
348000
351000
354000
357000
360000
363000
366000
369000
372000
375000
378000
381000
384000
387000
390000
393000
396000
399000
402000
405000
408000
411000
414000
417000
420000
423000
426000
429000
432000
435000
438000
441000
444000


# Train data split (Train & Valid)


In [26]:
# #train & test 데이터로 나누기 7:3
from sklearn.model_selection import train_test_split
                                                         
dataset_Train, dataset_Valid = train_test_split(data_list, test_size=0.3, random_state = 1)
print('dataset_Train :',len(dataset_Train))
print('dataset_Valid',len(dataset_Valid))

dataset_Train : 700000
dataset_Valid 300000


In [9]:
# Setting parameters
max_len = 32 
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1  
max_grad_norm = 1 
log_interval = 200
learning_rate =  5e-5 

In [8]:
#GPU 사용
device = torch.device("cuda:0")

#BERT 모델, Vocabulary 불러오기 

bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [10]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [11]:
#  BERTDataset class
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
#토큰화

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_Train = BERTDataset(dataset_Train, 0, 1, tok, max_len, True, False)
data_Valid = BERTDataset(dataset_Valid, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


NameError: ignored

In [ ]:
# DataLoader 
Train_dataloader = torch.utils.data.DataLoader(data_Train, batch_size=batch_size, num_workers=2)
Valid_dataloader = torch.utils.data.DataLoader(data_Valid, batch_size=batch_size, num_workers=2)

# Model 

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=len(class_name),   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(Train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
# train_dataloader

In [ ]:
num_epochs = 1
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        # token_ids = token_ids.long()
        segment_ids = segment_ids.long().to(device)
        # segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long().to(device)
        # label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #torch.save(model.state_dict(), '/content/drive/MyDrive/model_0412_km_{}.pth'.format('removemodel_22'))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        # label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/10938 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 5.685079574584961 train acc 0.0
epoch 1 batch id 201 loss 3.9605143070220947 train acc 0.16223569651741293
epoch 1 batch id 401 loss 1.594641089439392 train acc 0.42631701995012466
epoch 1 batch id 601 loss 0.663910448551178 train acc 0.568609608985025
epoch 1 batch id 801 loss 0.7442579865455627 train acc 0.6487398564294632
epoch 1 batch id 1001 loss 0.5820630192756653 train acc 0.6992382617382618
epoch 1 batch id 1201 loss 0.28422489762306213 train acc 0.7329699208992506
epoch 1 batch id 1401 loss 0.5557947754859924 train acc 0.7570596895074947
epoch 1 batch id 1601 loss 0.560264527797699 train acc 0.7751893347907558
epoch 1 batch id 1801 loss 0.36260679364204407 train acc 0.7900298445308163
epoch 1 batch id 2001 loss 0.5623184442520142 train acc 0.8019349700149925
epoch 1 batch id 2201 loss 1.0091991424560547 train acc 0.8123367219445706
epoch 1 batch id 2401 loss 0.23530274629592896 train acc 0.8211422324031653
epoch 1 batch id 2601 loss 0.6551896333694458 t

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/4688 [00:00<?, ?it/s]

epoch 1 test acc 0.9277110441552902


# Predict

In [ ]:
# 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/model_0412_km_remove_model_2.pth'))

In [6]:
import pandas as pd
submission_org =  pd.read_csv("/content/drive/MyDrive/통계데이터_인공지능_활용대회/답안 작성용 파일.csv", encoding='euc-kr')

In [7]:
submission_data = submission_org[['digit_3','text_obj','text_mthd','text_deal']]

In [8]:
# df_mod = df_mod.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(submission_data.isnull().values.any()) # Null 값이 존재하는지 확인
submission_data = submission_data.fillna(' ') # Nan 값을 빈 string으로 바꿈
print(submission_data.isnull().values.any()) # Null 값이 존재하는지 확인
# False => nan값이 없다

True
False


In [9]:
# data x, y 로 나누기
sub_x1 = submission_data['text_obj'].astype(str)
sub_x2 = submission_data['text_mthd'].astype(str)
sub_x3 = submission_data['text_deal'].astype(str)

In [ ]:
remove_1 = ['에서','으로','에게','하여',] # text_obj, text_mthd,
remove_2 =['일반','고객'] # text_mthd 
remove_3 = ['를 통하여','을 갖추고','를통하여'] #  text_mthd
for i in range(len(sub_x1)):
  # remove 1
  if sub_x1.iloc[i][-2::]in remove_1: # text_obj
    sub_x1[i]=sub_x1.iloc[i][0:-2]
  
  if sub_x2.iloc[i][-2::]in remove_1: # text_mthd
    sub_x2[i]=sub_x2.iloc[i][0:-2]

  # remove 2
  if sub_x2.iloc[i][0:2]in remove_2:  # text_mthd
    sub_x2[i]=' ' # remove_2 들어가면 제거 
    
  # remove 3
  if sub_x2.iloc[i][-5::]in remove_3: # text_mthd
    sub_x2[i]=sub_x2.iloc[i][0:-5]
  
  if sub_x2.iloc[i][-4::]in remove_3: # text_mthd
    sub_x2[i]=sub_x2.iloc[i][0:-4]
    
  if i % 10000 == 0:
    print(i)
    
    


In [11]:
sub_x = sub_x1 + ',' + sub_x2+','+ sub_x3

In [12]:
sub_x

0                      치킨전문점, ,치킨판매
1             산업공구,다른 소매업자에게,철물 수공구
2               절에서,신도을 대상으로,불교단체운영
3                 영업장에서,고객요구로,자동차튜닝
4        실내포장마차에서,접객시설을 갖추고,소주,맥주제공
                    ...            
99995            사업장에서,일반인대상으로,버섯농장
99996              한의원에서,외래환자위주고,치료
99997             일반점포에서,소비자에게,그림판매
99998       사업장에서,일반인.학생대상으로,학습공간제공
99999             사업장에서,대리현대아파트를,관리
Length: 100000, dtype: object

In [ ]:
stop_words = "에서 로 으로 갖추고 일반 대상 . , 등 따라 의해 에 로 위하여 ? 통해 하여 에게 소비 제공 소비자 고객 일반인 을 를 의 하여 해서 요청 가지고 및 두고 받아 받음 서비스"
stop_words = set(stop_words.split(' '))
print(stop_words)

{'의', ',', '으로', '일반인', '에서', '및', '대상', '서비스', '받음', '받아', '갖추고', '하여', '에게', '따라', '일반', '요청', '통해', '위하여', '소비자', '소비', '고객', '두고', '가지고', '.', '을', '로', '에', '제공', '?', '를', '등', '의해', '해서'}


In [ ]:
okt = Okt()

In [ ]:
q = sub_x[0]
word_tokens = okt.morphs(q)
print(word_tokens)
result = ' '
for word in word_tokens:
  if not word in stop_words:
    result += ' ' + word


['치킨', '전문점', '에서', '치킨', '판매']


In [ ]:
print(result)

  치킨 전문점 치킨 판매


In [ ]:
cnt = 0
for i in range(len(sub_x)):
  q = sub_x[i]
  word_tokens = okt.morphs(q)  
  result = ' '
  for word in word_tokens:
    if not word in stop_words:
      result += ' ' + word  
  sub_x[i] = result
  cnt += 1
  if cnt%1000 == 0:
    print(cnt)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):
    
    data = [predict_sentence, '0']
    dataset_another = [data]
    
    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        
        for i in out:
            logits=i
            
            logits = logits.detach().cpu().numpy()
            
            result = np.argmax(logits)
            print(class_name[result])
            test_eval.append(class_name[result])

            return class_name[result]
         



In [2]:
test_eval=[]
idx = len(x)

for i in range(idx):
  predict(sub_x[i])
  if i% 1000 == 0:

     print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000


In [ ]:
def search_big_middle(number):
  class_big = ''

  if number<51: class_big = 'A'
  elif number<101: class_big = 'B'
  elif number<351: class_big = 'C'
  elif number<360: class_big = 'D'
  elif number<411: class_big = 'E'
  elif number<451: class_big = 'F'
  elif number<491: class_big = 'G'
  elif number<551: class_big = 'H'
  elif number<581: class_big = 'I'
  elif number<641: class_big = 'J'
  elif number<681: class_big = 'K'
  elif number<701: class_big = 'L'
  elif number<741: class_big = 'M'
  elif number<841: class_big = 'N'
  elif number<851: class_big = 'O'
  elif number<861: class_big = 'P'
  elif number<901: class_big = 'Q'
  elif number<941: class_big = 'R'
  elif number<970: class_big = 'S'
  elif number<990: class_big = 'T'
  elif number<1000: class_big = 'U'
  return class_big

In [ ]:
#소분류 예측 (digit_3)
submission_org['digit_3'] = test_eval  

In [ ]:
# 중분류 예측 (digit_2)
submission_org['digit_2'] = submission_org['digit_3']//10 

In [ ]:
# 대분류 예측 (digit_1)
for i in range(len(submission_org)):
  submission_org['digit_1'][i] = search_big_middle(submission_org['digit_3'][i])
  if i % 5000 == 0:
    print(i)

In [ ]:
submission_org.to_csv('제출용_파일.csv', index=False)